In [1]:
import urllib
import json
import pandas as pd
import vertica_python
import re
from datetime import datetime, timedelta

In [4]:
x1 = datetime(2020,1,1)
x2 = datetime(2020,1,18)
d = []
while x1<=x2:
    d.append(x1.strftime('%Y-%m-%d'))
    x1 += timedelta(days=1)
    
print(d)

['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04', '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-11', '2020-01-12', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-18']


In [55]:
#Забираем файл и преобразовываем его в json.
url = 'https://www.cbr-xml-daily.ru//archive//2020//11//06//daily_json.js'
#url = 'https://www.cbr-xml-daily.ru/daily_json.js'
response = urllib.request.urlopen(url)
data = json.loads(response.read())

In [61]:
valute = ('USD', 'EUR', 'CNY', 'INR', 'JPY')
delimiter = '\x1F'
line_terminator = '\x1E'

In [62]:
table_header = "valute.valute(ID, NumCode, CharCode, Nominal, Name, Value)"
rej_table = 'valute.rej_table_valute'

In [2]:
connection_info = {'host': 'localhost',
             'port': 5433,
             'user': 'dbadmin',
             'password': '',
             # default throw error on invalid UTF-8 results
             'unicode_error': 'strict',
             # SSL is disabled by default
             'ssl': False,
             # autocommit is off by default
             'autocommit': True,
             # using server-side prepared statements is disabled by default
             'use_prepared_statements': False,
             # connection timeout is not enabled by default
             # 5 seconds timeout for a socket operation (Establishing a TCP connection or read/write operation)
             'connection_timeout': 5}


In [6]:
#This code only for experiment ma'faka
#with open('valute.csv', 'w', newline='', encoding='utf8') as csvfile:
    #data_writer = csv.writer(csvfile, delimiter = '\x1F', lineterminator = '\x1E')
    #for i in valute:
        #for j in atribute:
            #a = str(data['Valute'][i][j])
            #data_writer.writerow([a]) 
            #print(type(a), a)

In [10]:
with vertica_python.connect(**connection_info) as connection:
        cur = connection.cursor()
        cur.execute("START TRANSACTION;")
        cur.execute("SET SESSION AUTOCOMMIT TO OFF;")
        a = cur.execute('select count(*) from valute.valute;').fetchall()
        print(type(a[0][0]), '|' , a[0][0])

<class 'int'> | 34


In [51]:
type(data)

dict

ValueError: DataFrame constructor not properly called!

In [63]:
df = pd.DataFrame(data['Valute'])
current_date = data['Date']
df = df.T

In [64]:
type(current_date)

str

In [67]:
df.drop(['Previous'], axis = 1, inplace = True)
df['Nominal'] = df['Nominal'].astype(int)
df['Value'] = df['Value'].astype(float)
df['Date'] = re.match(r'\d{4}-\d{2}-\d{2}',current_date).group(0)

In [68]:
df

,ID,NumCode,CharCode,Nominal,Name,Value,Date
AUD,R01010,036,AUD,1,Австралийский доллар,56.3702,2020-11-03
AZN,R01020A,944,AZN,1,Азербайджанский манат,47.4249,2020-11-03
GBP,R01035,826,GBP,1,Фунт стерлингов Соединенного королевства,103.7080,2020-11-03
AMD,R01060,051,AMD,100,Армянских драмов,16.3554,2020-11-03
BYN,R01090B,933,BYN,1,Белорусский рубль,30.3027,2020-11-03
BGN,R01100,975,BGN,1,Болгарский лев,47.9156,2020-11-03
BRL,R01115,986,BRL,1,Бразильский реал,14.0304,2020-11-03
HUF,R01135,348,HUF,100,Венгерских форинтов,25.5651,2020-11-03
HKD,R01200,344,HKD,1,Гонконгский доллар,10.3918,2020-11-03
DKK,R01215,208,DKK,1,Датская крона,12.5853,2020-11-03


In [10]:
df.to_csv('valute.csv', index=False, sep = delimiter, line_terminator = line_terminator,
            header=True,
            encoding='utf-8',
            na_rep=' ')

In [11]:
with vertica_python.connect(**connection_info) as connection:
        copy_req = """COPY {table_header}
                FROM STDIN PARSER
                fcsvparser(
                    type='traditional',
                    delimiter='{delimiter}',
                    header='true',
                    record_terminator='{line_terminator}',
                    reject_on_materialized_type_error = 'true'
                ) REJECTED DATA AS TABLE {rej_table};
            """.format(
            table_header=table_header,
            delimiter=delimiter,
            line_terminator=line_terminator,
            rej_table=rej_table
        )
        cur = connection.cursor()
        cur.execute("START TRANSACTION;")
        cur.execute("SET SESSION AUTOCOMMIT TO OFF;")
        with open('C:\\Users\\adyan.beltikov\\Desktop\\programs\\Glob_currency\\valute.csv', "rb") as fs:
            try:
                cur.copy(copy_req, fs, buffer_size=65536)
                cur.execute('select get_num_rejected_rows();')
                after = cur.fetchone()
                print(after)
                cur.execute("COMMIT;")
            except Exception as ex:
                print(ex)

[0]
